<a href="https://colab.research.google.com/github/SarthakNarayan/DL-and-ML/blob/master/googlecolab/Implementing_Inception.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Imports Section

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F

##Inception Structure
![alt text](https://www.oreilly.com/library/view/python-deep-learning/9781789348460/assets/954ea76d-97e0-47b1-8cf6-7e1ba4601cbd.png)

There are many versions of inception we will be implementing the one mention in the link below<br/>
https://drive.google.com/open?id=1jftGgnGQVuM1ThsSn48EIk7GlSvqgtFE

In [3]:
# Implementing an inception block
class Inception(nn.Module):
    def __init__(self , input_channels):
        super(Inception , self).__init__()
        self.conv1 = nn.Conv2d(input_channels , 16 , stride=1 , kernel_size = 1)
        
        self.conv2 = nn.Conv2d(input_channels , 16 , stride=1 , kernel_size = 1)
        # padding of 2 so that dimensions of image remain same
        self.conv3 = nn.Conv2d(16 , 24 , stride=1 , kernel_size = 5 , padding = 2)
        
        # No need of padding for 1*1 convolutions since size always remains same
        self.conv4 = nn.Conv2d(input_channels , 16 , stride=1 , kernel_size = 1)
        self.conv5 = nn.Conv2d(16 , 24 , stride=1 , kernel_size = 3 , padding = 1)
        self.conv6 = nn.Conv2d(24 , 24 , stride=1 , kernel_size = 3 , padding = 1)
        
        self.conv7 = nn.Conv2d(input_channels , 24 , stride=1 , kernel_size = 1)
        
    def forward(self , x):
        out1 = self.conv1(x)
        
        out2 = self.conv2(x)
        out2 = self.conv3(out2)
        
        out3 = self.conv4(x)
        out3 = self.conv5(out3)
        out3 = self.conv6(out3)
        
        out4 = F.avg_pool2d(x , 3 , stride = 1 , padding = 1)
        out4 = self.conv7(out4)        
        
        # shapes of the outputs
        # While training and prediction remove the print statements
        print("Output 1 {}".format(out1.shape))
        print("Output 2 {}".format(out2.shape))
        print("Output 3 {}".format(out3.shape))
        print("Output 4 {}".format(out4.shape))
        
        # now we have to concat the outputs
        # Shapes of the tensor being concated should match
        outputs = [out1,out2,out3,out4]
        output = torch.cat(outputs , 1)
        print("Concatenated Output {}".format(output.shape))
        return output
    
inception = Inception(3)
x = torch.randn(1,3,32,32)
y = inception(x)

Output 1 torch.Size([1, 16, 32, 32])
Output 2 torch.Size([1, 24, 32, 32])
Output 3 torch.Size([1, 24, 32, 32])
Output 4 torch.Size([1, 24, 32, 32])
Concatenated Output torch.Size([1, 88, 32, 32])


##Viewing the model

In [0]:
!pip install onnx
dummy_input = torch.randn(1, 3, 32, 32)
torch.onnx.export(inception, dummy_input, "inception.onnx")
print("Model Exported for viewing")

##Using Inception in a full Network

In [8]:
class Net(nn.Module):
    def __init__(self):
        super(Net , self).__init__()
        
        self.conv1 = nn.Conv2d(3,10,stride = 1 , kernel_size = 3)
        self.inception1 = Inception(10)
        self.conv2 = nn.Conv2d(88,20,stride = 1 , kernel_size = 3)
        self.inception2 = Inception(20)
        self.fc = nn.Linear(3168,10)
        self.mp = nn.MaxPool2d(2)
        
    def forward(self , x):
        x = F.relu(self.mp(self.conv1(x)))
        x = self.inception1(x)
        x = F.relu(self.mp(self.conv2(x)))
        x = self.inception2(x)
        print(x.shape)
        # always do this step when going from convolutional to linear
        x = x.view(x.size(0) , -1)
        print(x.shape)
        x = self.fc(x)
        return x
    
net = Net()
x = torch.randn(1,3,32,32)
y = net(x)

Output 1 torch.Size([1, 16, 15, 15])
Output 2 torch.Size([1, 24, 15, 15])
Output 3 torch.Size([1, 24, 15, 15])
Output 4 torch.Size([1, 24, 15, 15])
Concatenated Output torch.Size([1, 88, 15, 15])
Output 1 torch.Size([1, 16, 6, 6])
Output 2 torch.Size([1, 24, 6, 6])
Output 3 torch.Size([1, 24, 6, 6])
Output 4 torch.Size([1, 24, 6, 6])
Concatenated Output torch.Size([1, 88, 6, 6])
torch.Size([1, 88, 6, 6])
torch.Size([1, 3168])


##Viewing the full model

In [0]:
dummy_input = torch.randn(1, 3, 32, 32)
torch.onnx.export(net, dummy_input, "net.onnx")
print("Model Exported for viewing")